In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report, roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('Covid_patients12.csv')

In [3]:
data

,EDLOSbin,EDLengthofStayInHrs,PatientRace,PatientSex,ArrivalMethod,EdDisposition,DischargeDisposition,AcuityLevel,PatientClass,Bed Assigned,...,HIV,Asthma,HTN,Transplant,LowIncome,obese,SBP,HR,RR,Spo2
0,1,10,Black,Male,Car,Admit,Expired,2 - Emergent,Inpatient,1,...,0,0,1,1,0,0,193,70,22,97
1,1,13,White,Male,Other,Admit,Home or Self Care,2 - Emergent,Inpatient,1,...,0,0,0,0,0,1,127,108,24,95
2,0,3,Black,Female,Car,Admit,Home or Self Care,2 - Emergent,Inpatient,1,...,0,0,0,0,1,1,113,109,20,94
3,1,14,Other,Male,Ambulance,Admit,Expired,1 - Immediate,Inpatient,1,...,0,0,1,0,0,1,139,111,37,83
4,1,18,Black,Male,Car,Admit,Home or Self Care,2 - Emergent,Inpatient,1,...,0,1,1,0,1,1,117,101,22,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,1,6,Black,Male,Other,Admit,Expired,2 - Emergent,Inpatient,1,...,0,0,1,0,1,1,116,86,17,95
532,1,19,White,Male,Ambulance,Admit,Home or Self Care,3 - Urgent,Inpatient,1,...,0,0,0,0,0,0,133,72,18,99
533,1,15,White,Male,Ambulance,Admit,Skilled Nursing Facility,1 - Immediate,Inpatient,1,...,0,0,1,0,0,0,194,79,24,96
534,0,1,White,Male,Ambulance,Discharge,Home or Self Care,2 - Emergent,Emergency,0,...,0,0,0,0,0,1,151,113,20,99


In [4]:
def temp(x):
    if x > 100.4:
        return "High"
    if x < 97:
        return "Low"
    return "Mild"

In [5]:
data.Temp.apply(temp)

0      Mild
1      High
2      High
3      Mild
4      Mild
       ... 
531    Mild
532    Mild
533     Low
534    Mild
535    Mild
Name: Temp, Length: 536, dtype: object

In [44]:
X = data.loc[:, ['O2_Flow', 'CKD', 'CHF', 'DM', 'HTN', 'Resp_failure', 'Registered Nurse','Sepsis', 'Stroke', 'ESI', 'Bed Assigned', 'Physician']]
X = pd.concat([X, pd.get_dummies(data.loc[:, 'Age Group'])], axis=1)
X = pd.concat([X, pd.get_dummies(data.loc[:, 'PrimaryChiefComplaint'])], axis=1)
#X = pd.concat([X, pd.get_dummies(data.loc[:, 'PatientRace'])], axis=1)
X = pd.concat([X, pd.get_dummies(data.Temp.apply(temp))], axis=1)
y = data.EDLOSbin

In [89]:
# oversample data:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(random_state=0)
X_resampled, y_resampled = oversample.fit_resample(X, y)

In [106]:
X_train, X_test, y_train, y_test = train_test_split(
    X.dropna(axis=1), y, test_size=0.25, stratify=y)

#ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = oversample.fit_resample(X_train, y_train)

In [107]:
from sklearn.ensemble import RandomForestClassifier

In [110]:
forest = RandomForestClassifier(random_state=0)
forest.fit(X_resampled, y_resampled)
print(classification_report(forest.predict(X_test), y_test))

              precision    recall  f1-score   support

           0       0.57      0.59      0.58        46
           1       0.78      0.77      0.78        88

    accuracy                           0.71       134
   macro avg       0.68      0.68      0.68       134
weighted avg       0.71      0.71      0.71       134

